In [1]:
# This notebook makes setolabo samples as test samples. 

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [7]:
import pandas as pd

# Load data
df_GRE = pd.read_excel('E:/MedBank/head_model/NGS/data_shared_by_Takeshi-san/PRJNA946800.xlsx')
df_seto = pd.read_csv('E:/MedBank/head_model/NGS/data_shared_by_Takeshi-san/expression_count_test_20240807.csv')

# Remove duplicates from df_GRE based on 'miRNA'
df_GRE = df_GRE.drop_duplicates(subset=['miRNA'], keep='first')

# Create labels for df_GRE (case = 0, control = 1)
columns = df_GRE.columns.tolist()
labels_GRE = []
for item in columns:
    if item.startswith('case'):
        labels_GRE.append(0)
    elif item.startswith('control'):
        labels_GRE.append(1)
    else:
        pass

# Set labels for df_seto as 0 (since all are cancerous)
labels_seto = [0 for _ in range(5)]

# Filter both datasets based on common 'miRNA' / 'gene_id'
features = df_GRE['miRNA'].tolist()
df_seto = df_seto[df_seto['gene_id'].isin(features)]
features = df_seto['gene_id']
df_GRE = df_GRE[df_GRE['miRNA'].isin(features)]

# Sort both DataFrames by 'miRNA' / 'gene_id' to align them
df_GRE = df_GRE.sort_values(by='miRNA')
df_seto = df_seto.sort_values(by='gene_id')

# Transpose the DataFrames and reset their index
df_GRE = df_GRE.T
df_seto = df_seto.T
df_GRE.reset_index(drop=True, inplace=True)
df_seto.reset_index(drop=True, inplace=True)

# Update the column names after transposing
df_seto.columns = df_seto.iloc[0].tolist()
df_seto = df_seto[1:]
df_GRE.columns = df_GRE.iloc[0].tolist()
df_GRE = df_GRE[1:]

# Assign labels to the GRE and seto datasets
df_GRE['labels'] = labels_GRE
df_seto['labels'] = labels_seto

# No train-test split, df_GRE is the full training set and df_seto is the testing set
df_train = df_GRE.iloc[:, :-1]  # Features of the full training set
df_test = df_seto.iloc[:, :-1]  # Features of the full testing set

# Create labels_train and labels_test
labels_train = df_GRE['labels'].tolist()
labels_test = df_seto['labels'].tolist()


In [8]:
# Define NGS version as a variable
folder_path = "E:/MedBank/head_model/NGS/data_shared_by_Takeshi-san/model_ready_folders/setolabo_samples_as_test/PRJNA946800"  # You can change this value as needed

# Paths for train and test feature vectors
file_path = f"{folder_path}/train/feature_vectors.csv"
df_train.to_csv(file_path, index=False, header=False)

file_path = f"{folder_path}/test/feature_vectors.csv"
df_test.to_csv(file_path, index=False, header=False)

##--------------------------------------------------------

# Paths for train and test labels
file_path = f"{folder_path}/train/labels.txt"
with open(file_path, 'w') as file:
    for label in labels_train:
        file.write(f"{label}\n")

file_path = f"{folder_path}/test/labels.txt"
with open(file_path, 'w') as file:
    for label in labels_test:
        file.write(f"{label}\n")

##--------------------------------------------------------

# Path for feature names
file_path = f"{folder_path}/feature_names.txt"
with open(file_path, 'w') as file:
    for label in features:
        file.write(f"{label}\n")


In [1]:
text = 'case/sample_analysis/ERR372264/ERR372264_sort.bam'
if text.startswith('case'):
    print(True)

True
